# A LangChain 1.0 Agent that can talk with a Database

## What Does This Code Do?

This **AI Agent** that can talk to a SQL database and answer questions about it. Think of it as a smart assistant that knows how to write and execute SQL queries to find information for you.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.utilities import SQLDatabase

# Fix: Specify the tables with correct capitalization to avoid error
db = SQLDatabase.from_uri(
    "sqlite:///Chinook.db",
    include_tables=['Artist', 'Album', 'Track', 'Customer', 'Invoice']  # Use exact names!
)

from langchain.tools import tool

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o-mini",
    tools=[sql_query]
)

from langchain.messages import HumanMessage

question = HumanMessage(content="How many artists are in the database?")

response = agent.invoke(
    {"messages": [question]}
)

print(response["messages"][-1].content)

There are a total of 275 artists in the database.


In [4]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content='How many artists are in the database?', additional_kwargs={}, response_metadata={}, id='a44c1043-fb99-48f1-aa2c-574f3aaea769'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 54, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CxsK2OGjvshUEOzupK1zdU6XOf9Hm', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bbc08-0840-7f43-b472-cb8b504a58b5-0', tool_calls=[{'name': 'sql_query', 'args': {'query': 'SELECT COUNT(*) as artist_count FROM artists;'}, 'id': 'call_uVd0W1RPOp66Abn9rXg7mzxw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 5

## Let's explain the previous code in simple terms

#### 1. Loading Environment Variables
```python
from dotenv import load_dotenv
load_dotenv()
```
- **What it does**: Loads secret keys (like your OpenAI API key) from a `.env` file
- **Why**: Keeps your API keys secure and separate from your code
- **Beginner tip**: The `.env` file contains `OPENAI_API_KEY=your-key-here`

---

#### 2. Connecting to the Database
```python
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(
    "sqlite:///Chinook.db",
    include_tables=['Artist', 'Album', 'Track', 'Customer', 'Invoice']
)
```
- **What it does**: Creates a connection to a SQLite database file called `Chinook.db`
- **`include_tables`**: Limits access to only these 5 specific tables for safety
- **Important**: The table names must match EXACTLY (capital letters matter!)
- **Beginner tip**: This is like opening a door to the database so we can query it

---

#### 3. Creating a Tool for the Agent
```python
from langchain.tools import tool

@tool
def sql_query(query: str) -> str:
    """Obtain information from the database using SQL queries"""
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"
```
- **`@tool` decorator**: This special marker tells LangChain "this function is a tool the agent can use"
- **The docstring** (`"""Obtain information..."""`): The AI reads this to understand what the tool does!
- **Function purpose**: Takes a SQL query string, runs it on the database, and returns results
- **Error handling**: If something goes wrong (like a bad SQL query), it catches the error and returns a helpful message
- **Beginner tip**: Tools are like giving the AI special abilities—in this case, the ability to query databases

---

#### 4. Creating the Agent
```python
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o-mini",
    tools=[sql_query]
)
```
- **`create_agent`**: LangChain 1.0's production-ready function for creating AI agents
- **`model="gpt-4o-mini"`**: Specifies which AI model to use (you can also pass a model instance for more control)
- **`tools=[sql_query]`**: Gives the agent access to our SQL query tool
- **Under the hood**: Creates a graph-based agent using LangGraph that follows the **ReAct pattern** (Reasoning + Acting)
- **Beginner tip**: The agent can now "think" about your question and decide when to use the SQL tool

---

#### 5. Asking a Question
```python
from langchain.messages import HumanMessage

question = HumanMessage(content="How many artists are in the database?")

response = agent.invoke(
    {"messages": [question]}
)
```
- **`HumanMessage`**: A special message format that represents what you (the human) are asking
- **`agent.invoke()`**: Runs the agent with your question
- **What happens internally**:
  1. The agent reads your question
  2. Thinks: "I need to query the database to answer this"
  3. Decides to use the `sql_query` tool
  4. Generates a SQL query (e.g., `SELECT COUNT(*) FROM Artist`)
  5. Gets the result
  6. Formulates a natural language answer

---

#### 6. Getting the Answer
```python
print(response["messages"][-1].content)
```
- **`response["messages"]`**: Contains the entire conversation (your question + agent's thoughts + final answer)
- **`[-1]`**: Gets the last message (the final answer)
- **`.content`**: Extracts just the text of that message
- **Output**: "There are a total of 275 artists in the database."

---

#### 7. Using pprint to have a wider view of the reponse object

```python
pprint(response["messages"])
```

Using `pprint` gives us a full view of the response object. This comprehensive information is very useful when we want to understand what is going on in full detail.

---

#### How the Agent Reasoning Loop Works

When you run the agent, here's what happens step by step:

1. **Agent receives question**: "How many artists are in the database?"

2. **Agent thinks** (ReAct pattern - Reasoning):
   - "I need data from a database"
   - "I have a sql_query tool that can help"
   - "I should use a COUNT query on the Artist table"

3. **Agent acts** (ReAct pattern - Acting):
   - Calls: `sql_query("SELECT COUNT(*) as artist_count FROM artists;")`
   - *Gets error: no such table "artists"*

4. **Agent observes** the error and thinks again:
   - "The table name is wrong, let me try 'artist' (singular)"

5. **Agent acts again**:
   - Calls: `sql_query("SELECT COUNT(*) as artist_count FROM artist;")`
   - *Gets result: `[(275,)]`*

6. **Agent formulates final answer**:
   - Converts the raw result into natural language
   - Returns: "There are a total of 275 artists in the database."

This is the power of agents—they can **reason, use tools, handle errors, and try again** until they solve the problem.


## How to run this code from Visual Studio Code
* Open Terminal.
* Make sure you are in the project folder.
* Make sure you have the poetry env activated.
* Enter and run the following command:
    * `python 019-sql-agent.py`